In [72]:
import numpy as np
import pandas as pd

import time
import pandas as pd 
import ast

# from visualisation import generate_map

c_df = pd.read_csv('data/combined_dataframes.csv')
c_df.tail()

,latitude,longitude,bright_ti4,acq_date,instrument,confidence,bright_ti5
708003,-10.16375,142.13250,[333.5],['2019-12-13'],['VIIRS'],['n'],[295.5]
708004,-10.16375,142.14375,[340.0],['2019-12-23'],['VIIRS'],['n'],[296.6]
708005,-10.16375,142.14750,"[341.7, 309.7]",['2019-12-23'],['VIIRS'],['n'],"[296.4, 274.4]"
708006,-10.09625,142.16250,[315.5],['2019-10-08'],['VIIRS'],['n'],[290.3]
708007,-9.39875,142.64625,"[367.0, 345.1]",['2019-12-02'],['VIIRS'],"['h', 'n']","[301.8, 301.9]"


In [75]:
# Convert the contents of the pandas array from strings looking like lists to actual lists
brightness_MODIS = c_df.loc[:,'bright_ti4'].apply(ast.literal_eval)
brightness_VIIRS = c_df.loc[:,'bright_ti5'].apply(ast.literal_eval)
instrument = c_df.loc[:,'instrument'].apply(ast.literal_eval)

In [80]:
risk = np.zeros(len(c_df.latitude))

In [82]:
# calculate the mean of each of the brightnesses
import statistics 
for i,list in enumerate(brightness_MODIS):
    risk[i] += statistics.mean(list)

In [83]:
for i,list in enumerate(brightness_VIIRS):
    risk[i] += statistics.mean(list)

In [84]:
# calculate the average brightness
for i,list in enumerate(risk):
    risk[i] = risk[i] / len(instrument[i]) # divide by the number of instruments i.e. mean of 1 or mean of 2
# risk = np.mean(brightness_MODIS, brightness_VIIRS) # does not work

In [85]:
c_df['Risk'] = risk
c_df

,latitude,longitude,bright_ti4,acq_date,instrument,confidence,bright_ti5,Risk
0,-43.11875,146.61375,[313.6],['2020-01-09'],['MODIS'],[68],[301.9],615.5
1,-42.83375,147.20250,[328.7],['2020-01-02'],['MODIS'],[85],[304.1],632.8
2,-42.83000,147.21750,[324.9],['2020-01-02'],['MODIS'],[75],[302.2],627.1
3,-42.60125,147.00375,[323.5],['2020-01-02'],['MODIS'],[66],[295.9],619.4
4,-42.59375,146.98500,[322.1],['2020-01-03'],['MODIS'],[62],[297.0],619.1
...,...,...,...,...,...,...,...,...
708003,-10.16375,142.13250,[333.5],['2019-12-13'],['VIIRS'],['n'],[295.5],629.0
708004,-10.16375,142.14375,[340.0],['2019-12-23'],['VIIRS'],['n'],[296.6],636.6
708005,-10.16375,142.14750,"[341.7, 309.7]",['2019-12-23'],['VIIRS'],['n'],"[296.4, 274.4]",611.1
708006,-10.09625,142.16250,[315.5],['2019-10-08'],['VIIRS'],['n'],[290.3],605.8


In [86]:
risk

array([615.5, 632.8, 627.1, ..., 611.1, 605.8, 657.9])

In [ ]:
#other stuff I tried
#  ast.literal_eval(c_df.bright_ti4[1])
# c_df[2:]= c_df['bright_ti4'].apply(ast.literal_eval)
# c_df['Risk']= c_df['bright_ti4'].apply(len)
# c_df['Risk']=np.average(c_df.bright_ti4[1:20]) + np.average(c_df.bright_ti5[1:20])
# c_df.head()
# np.average(c_df.bright_ti4[1:20])
# c_df.bright_ti4.aggregate(sum)
# sum(axis=1)
# c_df